In [2]:
%pylab inline

from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

import requests
import eventlet
import pandas as pd
import operator

from bs4 import BeautifulSoup
from sklearn.decomposition import PCA
from IMDBAPI import IMDB
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import LatentDirichletAllocation

# will work with this text
data_plot_1 = """ american boxer decides prospector thirty receive jewelers maids aragorn """

# will work with this text
data_plot_2 = """ gets weed union growers indulgent self boyhood firepower oskar """


Populating the interactive namespace from numpy and matplotlib


In [43]:
def plotcounts (a):
    temp=a
    temp.replace(".", " ").replace(",", "")
    temp=temp.split(" ")
    temp=map(lambda x:x.lower(),temp)
    words=[]
    for b in movie_words:
        if b in temp:
            words.append(b)
    return words

def worddummy (a):
    if m in a:
        return 1
    else:
        return 0

def countactors(a):
    temp.append(a.split(", "))
    return temp

def token(text):
    return(text.split("|"))

# sequence generator from the model
# seed_text - representation of user`s input words sequence
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # pre-pad sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
    return in_text

#mglearn
def print_topics2(topics, feature_names, sorting, topics_per_chunk=6,
                 n_words=20):
    for i in range(0, len(topics), topics_per_chunk):
        # for each chunk:
        these_topics = topics[i: i + topics_per_chunk]
        # maybe we have less than topics_per_chunk left
        len_this_chunk = len(these_topics)
        # print topic headers
        print(("topic {:<8}" * len_this_chunk).format(*these_topics))
        print(("-------- {0:<5}" * len_this_chunk).format(""))
        # print top n_words frequent words
        for i in range(n_words):           
            try:                
                print(("{:<14}" * len_this_chunk).format(
                    *feature_names[sorting[these_topics, i]]))                
            except:
                pass
        print("\n")
        
#mglearn
def topics_plot(topics, feature_names, sorting, topics_per_chunk=6,
                 n_words=20):
    top_list = []
    for i in range(0, len(topics), topics_per_chunk):
        # for each chunk:
        these_topics = topics[i: i + topics_per_chunk]
        # maybe we have less than topics_per_chunk left
        len_this_chunk = len(these_topics)

        for i in range(n_words):           
            try:                
                top_list.append(("{:<14}" * len_this_chunk).format(
                    *feature_names[sorting[these_topics, i]]))                
            except:
                pass
    return top_list

In [42]:

################ <IMDB films extractor> #################
URL = "http://www.imdb.com/chart/top"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html.parser')
entries=soup.findAll('div', class_="wlb_ribbon")
# getting top movies ids from imdb
movie_ids = []
for a in entries:
    movie_ids.append(a['data-tconst'])
#print("========= movie_ids:", movie_ids) 
         
header = 'http://www.omdbapi.com/?apikey=6be019fc&tomatoes=true&i='
movie_info = []
for i in movie_ids:
    url = header + i
#print("========= url:", url) 
    r = requests.get(url).json()
    movie = []
    for a in r.keys():
        movie.append(r[a])
    movie_info.append(movie)
columns = r.keys()
df = pd.DataFrame(movie_info, columns = columns)
#print("========= df:", df)

################ <Extrating from films plot the most valuable words> #################
plots = list(df['Plot'])
temp = ""
for p in plots:
        temp = temp + p
temp.replace(".", " ").replace(",", "")
temp = temp.split(" ")
temp = list(map(lambda x:x.lower(),temp))
#print("=== plot_temp:", temp)

# collect valuable for further groups generatig words
vect = CountVectorizer(max_features=10000, max_df=.15, stop_words="english").fit(temp)
X = vect.transform(temp)

# test result
feature_names = vect.get_feature_names()
# print("=== Number of features: {}".format(len(feature_names)))
# print("=== The first 20 features:\n{}".format(feature_names[:20]))
# print("=== Each 2000 feature:\n{}".format(feature_names[::2000]))

# Max and min groups number (if number is greater than MAX_GROUPS_NUMBER - groups won`t be dense, info become useless)
# Here is LDA (Latent Dirichlet allocation - Латентное размещение Дирихле) is used for clustering (building topic model)
MAX_GROUPS_NUMBER = 85
#MIN_GROUPS_NUMBER = 10
lda = LatentDirichletAllocation(n_components=MAX_GROUPS_NUMBER, learning_method="batch", max_iter=25, random_state=0)
document_topics = lda.fit_transform(X)

# Sorting of features acsending order 
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# Getting of features names
feature_names = np.array(vect.get_feature_names())
print(feature_names)
# Take some number of topics to view results
print_topics2(topics=range(20), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10)
print('==== ', topics_plot(topics=range(1), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10))
################ <Demo: taking films from first two groups> ################# 
# (!!!) Take into account: films duplications were removed
# sorting by weight document of the topic number 1
group1 = np.argsort(document_topics[:, 1])[::-1]
films_1 = []

for i in group1[:10]:
    films_1.append(df[df['Plot'].str.contains(temp[i])].Title.unique())

print("========= First cluster\n")
for i in set(map(tuple, films_1)):
    print(''.join(list(i)))
    
# sorting by weight document of the topic number 2
group2 = np.argsort(document_topics[:, 2])[::-1]
films_2 = []

for i in group2[:10]:
    films_2.append(df[df['Plot'].str.contains(temp[i])].Title.unique())

print("========= Second cluster\n")
for i in set(map(tuple, films_2)):
    print('\n'.join(list(i)))

['000' '10' '12' ... 'zero' 'zone' 'zubrowka']
topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
tries         abandoned     story         young         help          
past          wwii          officer       run           group         
heart         retired       legendary     local         lonely        
solve         conflict      violent       left          boat          
forebodings   driver        pirate        holy          gift          
jewelers      edge          waste         crisis        willing       
flee          farmboy       collecting    multiple      religion      
flame         pirate        casino        wing          longshoreman  
firing        turned        outwit        budget        trap          
firepower     lie           finds         low           flee          


topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      

In [49]:
# integer encode sequences of words
print('==== Plot 1 = ', topics_plot(topics=range(1), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10))
plot_1 = topics_plot(topics=range(1), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10)
print('=== Plot list = ', plot_1)
plot_string_all = plot_1[:len(plot_1)]
plot_string = plot_1[: 4]
print('=== Plot_line= ', plot_string)

tokenizer = Tokenizer()
tokenizer.fit_on_texts([plot_string_all])
encoded = tokenizer.texts_to_sequences([plot_string_all])[0]
vocab_size = len(tokenizer.word_index) + 1

print('Vocabulary Size: %d' % vocab_size)
# encode 2 words -> 1 word
sequences = list()

for i in range(2, len(encoded)):
    sequence = encoded[i-2:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

# configure network
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network - will print loss and accuracy of each training epoch
model.fit(X, y, epochs=500, verbose=2)

# TEST with PLOT
print('=== PLOT_1 \n', generate_seq(model, tokenizer, max_length-1, plot_string, 8))

# TEST with TEXT
# print(generate_seq(model, tokenizer, max_length-1, 'Jack and', 4))

==== Plot 1 =  ['tries         ', 'past          ', 'heart         ', 'solve         ', 'forebodings   ', 'jewelers      ', 'flee          ', 'flame         ', 'firing        ', 'firepower     ']
=== Plot list =  ['tries         ', 'past          ', 'heart         ', 'solve         ', 'forebodings   ', 'jewelers      ', 'flee          ', 'flame         ', 'firing        ', 'firepower     ']
=== Plot_line=  ['tries         ', 'past          ', 'heart         ', 'solve         ']
Vocabulary Size: 11
Total Sequences: 8
Max Sequence Length: 3
Epoch 1/500
 - 2s - loss: 2.3984 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 2.3966 - acc: 0.0000e+00
Epoch 3/500
 - 0s - loss: 2.3951 - acc: 0.1250
Epoch 4/500
 - 0s - loss: 2.3933 - acc: 0.1250
Epoch 5/500
 - 0s - loss: 2.3916 - acc: 0.2500
Epoch 6/500
 - 0s - loss: 2.3898 - acc: 0.3750
Epoch 7/500
 - 0s - loss: 2.3880 - acc: 0.3750
Epoch 8/500
 - 0s - loss: 2.3861 - acc: 0.5000
Epoch 9/500
 - 0s - loss: 2.3843 - acc: 0.5000
Epoch 10/500
 - 0s - los

Epoch 160/500
 - 0s - loss: 0.2645 - acc: 1.0000
Epoch 161/500
 - 0s - loss: 0.2536 - acc: 1.0000
Epoch 162/500
 - 0s - loss: 0.2432 - acc: 1.0000
Epoch 163/500
 - 0s - loss: 0.2332 - acc: 1.0000
Epoch 164/500
 - 0s - loss: 0.2236 - acc: 1.0000
Epoch 165/500
 - 0s - loss: 0.2144 - acc: 1.0000
Epoch 166/500
 - 0s - loss: 0.2056 - acc: 1.0000
Epoch 167/500
 - 0s - loss: 0.1972 - acc: 1.0000
Epoch 168/500
 - 0s - loss: 0.1891 - acc: 1.0000
Epoch 169/500
 - 0s - loss: 0.1814 - acc: 1.0000
Epoch 170/500
 - 0s - loss: 0.1740 - acc: 1.0000
Epoch 171/500
 - 0s - loss: 0.1669 - acc: 1.0000
Epoch 172/500
 - 0s - loss: 0.1602 - acc: 1.0000
Epoch 173/500
 - 0s - loss: 0.1538 - acc: 1.0000
Epoch 174/500
 - 0s - loss: 0.1476 - acc: 1.0000
Epoch 175/500
 - 0s - loss: 0.1418 - acc: 1.0000
Epoch 176/500
 - 0s - loss: 0.1362 - acc: 1.0000
Epoch 177/500
 - 0s - loss: 0.1309 - acc: 1.0000
Epoch 178/500
 - 0s - loss: 0.1258 - acc: 1.0000
Epoch 179/500
 - 0s - loss: 0.1209 - acc: 1.0000
Epoch 180/500
 - 0s 

 - 0s - loss: 0.0083 - acc: 1.0000
Epoch 328/500
 - 0s - loss: 0.0082 - acc: 1.0000
Epoch 329/500
 - 0s - loss: 0.0082 - acc: 1.0000
Epoch 330/500
 - 0s - loss: 0.0081 - acc: 1.0000
Epoch 331/500
 - 0s - loss: 0.0080 - acc: 1.0000
Epoch 332/500
 - 0s - loss: 0.0080 - acc: 1.0000
Epoch 333/500
 - 0s - loss: 0.0079 - acc: 1.0000
Epoch 334/500
 - 0s - loss: 0.0078 - acc: 1.0000
Epoch 335/500
 - 0s - loss: 0.0078 - acc: 1.0000
Epoch 336/500
 - 0s - loss: 0.0077 - acc: 1.0000
Epoch 337/500
 - 0s - loss: 0.0077 - acc: 1.0000
Epoch 338/500
 - 0s - loss: 0.0076 - acc: 1.0000
Epoch 339/500
 - 0s - loss: 0.0075 - acc: 1.0000
Epoch 340/500
 - 0s - loss: 0.0075 - acc: 1.0000
Epoch 341/500
 - 0s - loss: 0.0074 - acc: 1.0000
Epoch 342/500
 - 0s - loss: 0.0074 - acc: 1.0000
Epoch 343/500
 - 0s - loss: 0.0073 - acc: 1.0000
Epoch 344/500
 - 0s - loss: 0.0072 - acc: 1.0000
Epoch 345/500
 - 0s - loss: 0.0072 - acc: 1.0000
Epoch 346/500
 - 0s - loss: 0.0071 - acc: 1.0000
Epoch 347/500
 - 0s - loss: 0.0071

Epoch 495/500
 - 0s - loss: 0.0030 - acc: 1.0000
Epoch 496/500
 - 0s - loss: 0.0030 - acc: 1.0000
Epoch 497/500
 - 0s - loss: 0.0030 - acc: 1.0000
Epoch 498/500
 - 0s - loss: 0.0030 - acc: 1.0000
Epoch 499/500
 - 0s - loss: 0.0030 - acc: 1.0000
Epoch 500/500
 - 0s - loss: 0.0029 - acc: 1.0000
=== PLOT_1 
 ['tries         ', 'past          ', 'heart         ', 'solve         ', ' ', 'f', 'o', 'r', 'e', 'b', 'o', 'd', 'i', 'n', 'g', 's', ' ', ' ', ' ', ' ', 'f', 'o', 'r', 'e', 'b', 'o', 'd', 'i', 'n', 'g', 's', ' ', ' ', ' ', ' ', 'f', 'o', 'r', 'e', 'b', 'o', 'd', 'i', 'n', 'g', 's', ' ', ' ', ' ', ' ', 'f', 'o', 'r', 'e', 'b', 'o', 'd', 'i', 'n', 'g', 's', ' ', ' ', ' ', ' ', 'f', 'o', 'r', 'e', 'b', 'o', 'd', 'i', 'n', 'g', 's', ' ', ' ', ' ', ' ', 'f', 'o', 'r', 'e', 'b', 'o', 'd', 'i', 'n', 'g', 's', ' ', ' ', ' ', ' ', 'f', 'o', 'r', 'e', 'b', 'o', 'd', 'i', 'n', 'g', 's', ' ', ' ', ' ', ' ', 'f', 'o', 'r', 'e', 'b', 'o', 'd', 'i', 'n', 'g', 's', ' ', ' ', ' ']
